In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import sys
import seaborn as sns

%matplotlib inline

In [2]:
ROOT_DATA = './data/'

In [4]:
!ls 'data'

ideas.txt
zoohackathon_test.zip
zoohackathon_train
zoohackathon_train.zip
zoohackathon_train_annot
zoohackathon_train_annot.zip


In [8]:
train_data_series = os.listdir(os.path.join(ROOT_DATA, 'zoohackathon_train'))

In [6]:
data_series

['ideas.txt',
 'zoohackathon_test.zip',
 'zoohackathon_train',
 'zoohackathon_train.zip',
 'zoohackathon_train_annot',
 'zoohackathon_train_annot.zip']